# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [54]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key



# Google developer API key
#from config import gkey

# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [27]:



vacation_df = pd.read_csv('city_df.csv')
dropna_vacation_df = vacation_df.dropna()
del dropna_vacation_df["Unnamed: 0"]
dropna_vacation_df.head(20)



,city,latitude,longitude,Max_Temp,humidity,cloudness,wind speed,country,Date
0,punta arenas,-53.1500,-70.9167,41.11,81,40,20.71,CL,1659350685
1,iqaluit,63.7506,-68.5145,47.93,87,75,2.30,CA,1659350685
2,yablonovo,50.8523,37.3574,79.95,39,42,12.24,RU,1659350685
3,bitung,1.4451,125.1824,81.45,81,49,3.62,ID,1659350685
4,carutapera,-1.1950,-46.0200,77.40,92,63,6.64,BR,1659350685
5,leningradskiy,69.3833,178.4167,41.85,81,35,7.94,RU,1659350685
6,puerto ayora,-0.7393,-90.3518,69.55,96,100,8.01,EC,1659350685
7,busselton,-33.6500,115.3333,55.11,73,100,33.49,AU,1659350685
8,negombo,7.2083,79.8358,80.65,78,40,6.91,LK,1659350686
9,saldanha,-33.0117,17.9442,62.65,51,0,9.51,ZA,1659350686


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [56]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Locations
locations = dropna_vacation_df[["latitude", "longitude"]]

humidity = dropna_vacation_df["humidity"].astype(float)


In [57]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig 

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [58]:
# Zero cloudiness:
cloud_df = dropna_vacation_df[dropna_vacation_df["cloudness"] == 0]

# A max temperature lower than 80 degrees but higher than 70:
temp_df = cloud_df[(cloud_df["Max_Temp"] > 70) & (cloud_df["Max_Temp"] < 80)]

# Wind speed less than 10 mph:
wind_df = temp_df[temp_df["wind speed"] < 10]

# Drop any rows will null values:
wind_df = wind_df.dropna()
wind_df



,city,latitude,longitude,Max_Temp,humidity,cloudness,wind speed,country,Date
60,laguna,38.4210,-121.4238,72.18,70,0,8.05,US,1659350647
163,vicksburg,32.3526,-90.8779,74.93,97,0,4.61,US,1659350699
193,ceres,37.5949,-120.9577,73.72,62,0,6.91,US,1659350703
473,springfield,37.2153,-93.2982,71.92,94,0,6.91,US,1659350587
475,riverbank,37.7360,-120.9355,74.52,66,0,6.91,US,1659350735


In [59]:
vaca_locations = wind_df[["latitude", "longitude"]]

vaca_humidity = wind_df["humidity"].astype(float)

# Plot Heatmap
vaca_fig = gmaps.figure()

# Create heat layer
vaca_heat_layer = gmaps.heatmap_layer(vaca_locations, weights=vaca_humidity, 
                                 dissipating=False, max_intensity=50,
                                 point_radius=2.5)

# Add layer
vaca_fig.add_layer(vaca_heat_layer)

# Display figure
vaca_fig


Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [60]:
# Store into variable named hotel_df:
hotel_df = wind_df

# Add a "Hotel Name" column to the DataFrame: (setting new columns to hold values)
hotel_df["Hotel Name"] = ""

# Set parameters to search for hotels with 5000 meters:

# Set params:
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Loop through to identify hotels:
for index, row in hotel_df.iterrows():
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat = row["latitude"]
    lng = row["longitude"]
    
    params['location'] = f'{lat},{lng}'
    
    hotel_data = requests.get(base_url, params=params).json()
    
    try:
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data['results'][0]['name']
        
    except: 
        print('Issues with finding a hotel at', row['city'])
        hotel_df.loc[index, "Hotel Name"] = "NA"
    
hotel_df

Issues with finding a hotel at laguna
Issues with finding a hotel at vicksburg
Issues with finding a hotel at ceres
Issues with finding a hotel at springfield
Issues with finding a hotel at riverbank


,city,latitude,longitude,Max_Temp,humidity,cloudness,wind speed,country,Date,Hotel Name
60,laguna,38.4210,-121.4238,72.18,70,0,8.05,US,1659350647,NA
163,vicksburg,32.3526,-90.8779,74.93,97,0,4.61,US,1659350699,NA
193,ceres,37.5949,-120.9577,73.72,62,0,6.91,US,1659350703,NA
473,springfield,37.2153,-93.2982,71.92,94,0,6.91,US,1659350587,NA
475,riverbank,37.7360,-120.9355,74.52,66,0,6.91,US,1659350735,NA


In [61]:
# Checking hotels:
len(hotel_df[["latitude", "longitude"]])


5

In [62]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>city</dt><dd>{city}</dd>
<dt>country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]

In [63]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

Figure(layout=FigureLayout(height='420px'))